# Imports

In [28]:
import pandas as pd
import requests
import base64
from IPython.display import display, Javascript

# Access token requisition

## Variables

In [29]:
client_id = 'c6ca0e7ef3d541d187ea9bbda30f9304'
client_secret = '29127565b24644959312ba8c3eab8c8e'

## Base 64 conversion

In [30]:
string = client_id + ':' + client_secret
string_bytes = string.encode('ascii')
base64_bytes = base64.b64encode(string_bytes)
base64_string = base64_bytes.decode('ascii')

## Requisition

In [31]:
url = 'https://accounts.spotify.com/api/token'
headers = {'Authorization': f'Basic {base64_string}',
           'Content-Type': 'application/x-www-form-urlencoded'}
payload = {'grant_type': 'client_credentials'}
response = requests.request('POST', url = url, headers = headers, data = payload)
access_token = response.json()['access_token']

# Artists dataframe

In [19]:
def show_alert(message):
    display(Javascript(f'alert("{message}")'))

def get_artist_data(artist_id, access_token):
    url = f'https://api.spotify.com/v1/artists/{artist_id}'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        show_alert(f'Error {response.status_code}: {response.json().get("error", {}).get("message", "Erro ao alimentar a tabela df_artist.")}')
        return None
    return response.json()

artist_ids = ['10gzBoINW3cLJfZUka8Zoe','4q3ewBCX7sLwd24euuV69X','25uiPmTg16RbhZWAqwLBy5','6M2wZ9GZgrQXHCFfjv46we','0X2BH1fck6amBIoJhDVmmJ']

artist_data = []
for artist_id in artist_ids:
    data = get_artist_data(artist_id, access_token)
    if data is not None:
        artist_data.append(data)

df_artists = pd.DataFrame(columns=['id', 'name', 'followers_total', 'genres', 'popularity'])

for artist in artist_data:
    new_row = pd.DataFrame({
        'id': [artist['id']],
        'name': [artist['name']],
        'followers_total': [artist['followers']['total']],
        'genres': [artist['genres']],
        'popularity': [artist['popularity']]
    })
    df_artists = pd.concat([df_artists, new_row], ignore_index=True)

df_artists

,id,name,followers_total,genres,popularity
0,10gzBoINW3cLJfZUka8Zoe,Above & Beyond,797104,"[edm, progressive house, progressive trance, t...",59
1,4q3ewBCX7sLwd24euuV69X,Bad Bunny,82027340,"[reggaeton, trap latino, urbano latino]",93
2,25uiPmTg16RbhZWAqwLBy5,Charli xcx,3223673,"[art pop, candy pop, metropopolis, pop, uk pop]",79
3,6M2wZ9GZgrQXHCFfjv46we,Dua Lipa,43615129,"[dance pop, pop, uk pop]",87
4,0X2BH1fck6amBIoJhDVmmJ,Ellie Goulding,12409247,"[indietronica, metropopolis, pop, uk pop]",77


# Albums dataframe

In [32]:
def show_alert(message):
    display(Javascript(f'alert("{message}")'))

def get_album_data(artist_id, access_token):
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups=album&market=BR'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        show_alert(f'Error {response.status_code}: {response.json().get("error", {}).get("message", "Erro ao alimentar a tabela df_album.")}')
        return None
    return response.json()

artist_ids = ['10gzBoINW3cLJfZUka8Zoe','4q3ewBCX7sLwd24euuV69X','25uiPmTg16RbhZWAqwLBy5','6M2wZ9GZgrQXHCFfjv46we','0X2BH1fck6amBIoJhDVmmJ']

albums_data = []

for artist_id in artist_ids:
    data = get_album_data(artist_id, access_token)
    if data and 'items' in data:
        for item in data['items']:
            total_tracks = item.get('total_tracks')
            album_id = item.get('id')
            name_album = item.get('name')
            release_date = item.get('release_date')
            artists = ', '.join(artist['name'] for artist in item['artists'])

            albums_data.append({
                'artist_id': artist_id,
                'album_id': album_id,
                'total_tracks': total_tracks,
                'name_album': name_album,
                'release_date': release_date,
                'artists': artists
            })

df_albums = pd.DataFrame(albums_data, columns=['artist_id', 'album_id', 'total_tracks', 'name_album', 'release_date', 'artists'])

df_albums

,artist_id,album_id,total_tracks,name_album,release_date,artists
0,10gzBoINW3cLJfZUka8Zoe,2Zrw977AKsXRDtKO8JETDP,24,Group Therapy 582 (DJ Mix),2024-06-08,"Above & Beyond, Above & Beyond Group Therapy, ..."
1,10gzBoINW3cLJfZUka8Zoe,3I5rkd5mQLJ28v9FNvLuSo,36,Group Therapy 581 (DJ Mix),2024-06-01,"Above & Beyond Group Therapy, Anjunabeats"
2,10gzBoINW3cLJfZUka8Zoe,5z8dhq8yJBAkniVdMHygZ5,35,Group Therapy 580 (DJ Mix),2024-05-25,"Above & Beyond, Above & Beyond Group Therapy, ..."
3,10gzBoINW3cLJfZUka8Zoe,3tGEphuLjzGHgns2h6yjDZ,35,Group Therapy 579 (DJ Mix),2024-05-18,"Above & Beyond, Above & Beyond Group Therapy, ..."
4,10gzBoINW3cLJfZUka8Zoe,6yfCLTc2NsdQQUwx9Fmgwl,25,The Anjunadeep Edition 500 with Above & Beyond...,2024-05-16,Above & Beyond
5,10gzBoINW3cLJfZUka8Zoe,5j6M828HP3XrXQPo59qyzr,35,OceanLab: The Anthology,2024-03-15,"Above & Beyond, OceanLab"
6,10gzBoINW3cLJfZUka8Zoe,0igLyZJXIkjl4DwDYFovhs,29,Group Therapy 500 Live from Los Angeles,2024-02-29,Above & Beyond
7,10gzBoINW3cLJfZUka8Zoe,5yqu78M4yqUQpyZGgrqAy3,12,Group Therapy 500 Live From Los Angeles - Deep...,2022-10-16,"Above & Beyond, Above & Beyond Group Therapy, ..."
8,10gzBoINW3cLJfZUka8Zoe,5U5RVS6wdPaleEFqBzzpDm,35,Anjunabeats Volume 16,2022-07-22,"Above & Beyond, Anjunabeats"
9,10gzBoINW3cLJfZUka8Zoe,6GhXirWzqrTaGIdi1r7sGi,11,The Last Glaciers (Original Motion Picture Sou...,2022-03-22,Above & Beyond


# Top Tracks dataframe

In [33]:
def show_alert(message):
    display(Javascript(f'alert("{message}")'))

def get_top_track_data(artist_id, access_token):
    url = f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks?market=BR'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        show_alert(f'Error {response.status_code}: {response.json().get("error", {}).get("message", "Erro ao alimentar a tabela df_top_tracks.")}')
        return None
    return response.json()['tracks']

artist_ids = ['10gzBoINW3cLJfZUka8Zoe', '4q3ewBCX7sLwd24euuV69X', '25uiPmTg16RbhZWAqwLBy5', '6M2wZ9GZgrQXHCFfjv46we', '0X2BH1fck6amBIoJhDVmmJ']
track_data = []

for artist_id in artist_ids:
    top_tracks = get_top_track_data(artist_id, access_token)
    if top_tracks is None:
        continue

    for track in top_tracks:
        track_id = track['id']
        track_name = track['name']
        track_duration_ms = track['duration_ms']
        track_popularity = track['popularity']
        track_release_date = track['album']['release_date']

        artists_names = ', '.join([artist['name'] for artist in track['artists']])

        track_data.append({
            'id_artist': artist_id,
            'artists': artists_names,
            'id_album': track['album']['id'],
            'name_album': track['album']['name'],
            'id_track': track_id,
            'name_track': track_name,
            'duration_ms': track_duration_ms,
            'popularity_track': track_popularity,
            'release_date_track': track_release_date
        })

df_top_tracks = pd.DataFrame(track_data, columns=['id_artist', 'artists', 'id_album', 'name_album', 'id_track', 'name_track', 'duration_ms', 'popularity_track', 'release_date_track'])

df_top_tracks

,id_artist,artists,id_album,name_album,id_track,name_track,duration_ms,popularity_track,release_date_track
0,10gzBoINW3cLJfZUka8Zoe,Above & Beyond,743cLwhIfyy5dFZNEF6zOw,Blue Monday,5dXZhZbDHWu6zwXkssFQcs,Blue Monday,252919,51,2020-01-20
1,10gzBoINW3cLJfZUka8Zoe,Above & Beyond,3tGEphuLjzGHgns2h6yjDZ,Group Therapy 579 (DJ Mix),4ny4zmSTcyxNMN532zIKZT,Group Therapy Intro (ABGT579) - Mixed,30961,33,2024-05-18
2,10gzBoINW3cLJfZUka8Zoe,"Above & Beyond, Marsh",7GGVUzqEZQFwbHsBO3Fzam,Believer (Marsh's Guatape Remix),1GHOLip5jcgRIBqd8QUx7N,Believer - Marsh's Guatape Remix,255778,46,2022-01-06
3,10gzBoINW3cLJfZUka8Zoe,"Above & Beyond, anamē, Marty Longstaff",0wyLwBTTorS8W4yHXJNPGj,Gratitude,0KU8W0lHfsNlH7lfV1dz29,Gratitude,233619,45,2022-04-14
4,10gzBoINW3cLJfZUka8Zoe,"Above & Beyond, Richard Bedford",2I6sty06i8pXtu175kAfHa,Group Therapy,4rfhAoZjGwraqwX1w47uij,Sun & Moon - Original Mix,326266,45,2011-07-04
5,10gzBoINW3cLJfZUka8Zoe,"Above & Beyond, Zoë Johnston",6coJxgM3dkzZbbuPBVVcOW,We Are All We Need,7jji8nfxJ84AJkUSpgruPL,We're All We Need,262426,44,2015-01-19
6,10gzBoINW3cLJfZUka8Zoe,"Above & Beyond, Zoë Johnston",7nin9VgWvgxpibL3zxc5gR,Crazy Love,273cv9sESQzXxMqEkhXdm4,Crazy Love,231497,44,2024-02-09
7,10gzBoINW3cLJfZUka8Zoe,"Above & Beyond, OceanLab, Marsh",5j6M828HP3XrXQPo59qyzr,OceanLab: The Anthology,3kwIZKjCkubNrPQ5mvxhYF,Sirens Of The Sea - Marsh Remix,271138,39,2024-03-15
8,10gzBoINW3cLJfZUka8Zoe,"Seven Lions, Above & Beyond, Opposite the Other",7udk1qiys7QiEPKN9lY8a4,Over Now,606Z051kpzw0rrcBCYCTtR,Over Now,175635,42,2023-09-22
9,10gzBoINW3cLJfZUka8Zoe,"Above & Beyond, Zoë Johnston",4DFeEGmEGdfnaUWwXBgjeY,A State of Trance 2024 (Mixed by Armin van Buu...,1QcJHSrj0PDx8Bo6dPHToY,Crazy Love (Mixed),276093,42,2024-05-17
